Extract mel-spectrogram from audio

In [1]:
import glob
import numpy as np
from sklearn.model_selection import KFold
from keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

Find graphic specs

In [ ]:
 # Get file paths
pd_files = glob.glob("./plots/PD/*.png")
hc_files = glob.glob("./plots/HC/*.png")

train

In [ ]:
def convert_images_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert image to numpy array
        image = load_img(file, target_size=(640, 480))
        image_arr = img_to_array(image)
        images_as_array.append(image_arr)
    return np.array(images_as_array)

In [ ]:
pd_images = convert_images_to_array(pd_files)
hc_images = convert_images_to_array(hc_files)

# Flatten images
pd_images = pd_images.reshape((pd_images.shape[0], -1))
hc_images = hc_images.reshape((hc_images.shape[0], -1))

# Create labels
pd_labels = np.ones(len(pd_files))
hc_labels = np.zeros(len(hc_files))

# Concatenate data and labels
data = np.concatenate((pd_images, hc_images), axis=0)
labels = np.concatenate((pd_labels, hc_labels), axis=0)

# Split data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)


ValueError: cannot reshape array of size 0 into shape (0,newaxis)

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)

accuracy_values = []
precision_values = []
recall_values = []
loss_values = []

for train_index, test_index in kfold.split(data, labels):
    model = xgb.XGBClassifier(use_label_encoder=False)
    model.fit(data[train_index], labels[train_index])
    
    predictions = model.predict(data[test_index])
    probabilities = model.predict_proba(data[test_index])

    accuracy = accuracy_score(labels[test_index], predictions)
    precision = precision_score(labels[test_index], predictions)
    recall = recall_score(labels[test_index], predictions)
    loss = log_loss(labels[test_index], probabilities)
    
    accuracy_values.append(accuracy)
    precision_values.append(precision)
    recall_values.append(recall)
    loss_values.append(loss)
    
    print("Accuracy: %.2f%%" % (accuracy * 100))
    print("Precision: %.2f%%" % (precision * 100))
    print("Recall: %.2f%%" % (recall * 100))
    print("Log Loss: %.2f" % loss)

C:\Users\datng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 72.73%
Precision: 62.50%
Recall: 100.00%
Log Loss: 0.83


C:\Users\datng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 80.00%
Precision: 100.00%
Recall: 66.67%
Log Loss: 0.64


C:\Users\datng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 100.00%
Precision: 100.00%
Recall: 100.00%
Log Loss: 0.11


C:\Users\datng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 100.00%
Precision: 100.00%
Recall: 100.00%
Log Loss: 0.13


C:\Users\datng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 90.00%
Precision: 80.00%
Recall: 100.00%
Log Loss: 0.14


In [ ]:
# Final performance report
print("\nFinal Cross-Validation Metrics:")
print("Accuracy: %.2f%%" % (np.mean(accuracy_values) * 100))
print("Precision: %.2f%%" % (np.mean(precision_values) * 100))
print("Recall: %.2f%%" % (np.mean(recall_values) * 100))
print("Log Loss: %.2f" % np.mean(loss_values))


Final Cross-Validation Metrics:
Accuracy: 88.55%
Precision: 88.50%
Recall: 93.33%
Log Loss: 0.37


In [ ]:
model.save_model('xgboost.model')